Learning to rank with scikit-learn: the pairwise transform
============================================================
На основе семинара Никиты Волкова и https://habrahabr.ru/post/263823/

**Key words:** 
   задача ранжированя, построение признаков для задачи ранжирования, попарный и списочный подходы, Lemur Project, обратный индекс.


<h3> Plan </h3>
  * **HW6 discussion** (10 minutes)
     
  * **Features for ranking ** (20 minutes)
  * **RankLib (The Lemur Project)** (40 minutes)
  * **Reverse index** (10 minutes)</span> 

# Features for ranking

### TF-IDF - классический текстовый признак


TF-IDF$(q,d)$ - мера релевантности документа $d$ запросу $q$

$n_{dw}$ (term frequency) - число вхождений слова $w$ в текст $d$;

$N_{w}$ (document frequency) - число документов, содержащих~$w$;

$N$ - число документов в коллекции $D$;


$N_{w}/N$ - оценка вероятности встретить слово $w$ в документе;

$(N_{w}/N)^{n_{dw}}$ - оценка вероятности встретить его $n_{dw}$ раз;

$P(q,d) = \prod\limits_{w\in q} (N_{w}/N)^{n_{dw}}$ -
оценка вероятности встретить в~документе $d$ слова запроса $q=\{w_1,\dots,w_k\}$ **чисто случайно**;

Оценка релевантности запроса $q$ документу $d$:

$$ -\log P(q,d) = \sum_{w\in q}
        \underbrace{n_{dw}\mathstrut}_{\text{TF}(w,d)}
        \underbrace{\log (N/N_{w})}_{\text{IDF}(w)}
        \;\; \to \;\; \max. $$

TF$(w,d) = n_{dw}$ - term frequency;

IDF$(w)=\log (N/N_{w})$ - inverted document frequency.

### PageRank - классический ссылочный признак

Документ $d$ тем важнее,
- чем больше других документов $c$ ссылаются на $d$,
- чем важнее документы $c$, ссылающиеся на $d$,
- чем меньше других ссылок имеют эти документы $c$.

    
Вероятность попасть на страницу $d$, если кликать случайно:
    
$$PR(d) = \frac{1-\delta}N + \delta\! \sum_{c\in D^{in}_d} \frac{PR(c)}{|D^{out}_c|},$$
    
    
$D^{in}_d \subset D$ - множество документов, ссылающихся на $d$,

$D^{out}_c \subset D$ - множество документов, на которые ссылается $c$,

$\delta = 0.85$ - вероятность продолжать клики (damping factor),

$N$ - число документов в коллекции $D$.

Sergey Brin, Lawrence Page.
The Anatomy of a Large-Scale Hypertextual Web Search Engine. 1998.

** Упражнение:** Посчитать PageRank для небольшого графа.

In [1]:
import itertools
import numpy as np
from scipy import stats, linalg
from sklearn import svm, linear_model, cross_validation

import matplotlib.pyplot as plt
%matplotlib inline

plt.rc('text', usetex=True)
plt.rc('text.latex', unicode=True)
plt.rc('text.latex', preamble='\\usepackage[utf8]{inputenc}')
plt.rc('text.latex', preamble='\\usepackage[russian]{babel}')
plt.rc('font', family='serif', size='16')

/Users/flaren/anaconda/envs/MyPython2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# RankLib (The Lemur Project)

<img src="./lemur-big-370.png">

RankLib --- мощная библиотека для обучения ранжированию. Написана на Java. Описание https://sourceforge.net/p/lemur/wiki/RankLib/

-**MART** (градиентный бустинг над регрессионными деревьями) [6]

-**RankNet** [1] (попарный подход, гладкий функционал качества, метод стохастического градиента)

-**RankBoost** [2] (модификация AdaBoost для классификации пар документов)

-**AdaRank** [3] (листовой подход, бустинг, базовые ранкеры - признаки, экспоненциальная функция)

-**Coordinate Ascent** (метода координатного подъема) [4]

-**LambdaMART** [5] (попарный/листовой, комбинация MART и LambdaRank)

-**ListNet** [7] (листовой подход, оптимизация гладкой функции, похожей на целевую, сравниваем распределения истинных меток с порождаемыми)

-**Random Forests** [8]

Возьмем датасет с конкурса «Интернет-математика 2009»
https://academy.yandex.ru/events/data_analysis/grant2009/

Ниже определены некоторые вспомогательные функции.

#### Особенности формата данных:

<line> .=. <target> qid:<qid> <feature>:<value> <feature>:<value> ... <feature>:<value> # <info>
<target> .=. <positive integer> метка релевантности
<qid> .=. <positive integer> номер запроса
<feature> .=. <positive integer>
<value> .=. <float>
<info> .=. <string>

Пример:

3 qid:1 1:1 2:1 3:0 4:0.2 5:0 # 1A
2 qid:1 1:0 2:0 3:1 4:0.1 5:1 # 1B 
1 qid:1 1:0 2:1 3:0 4:0.4 5:0 # 1C
1 qid:1 1:0 2:0 3:1 4:0.3 5:0 # 1D  
1 qid:2 1:0 2:0 3:1 4:0.2 5:0 # 2A  
2 qid:2 1:1 2:0 3:1 4:0.4 5:0 # 2B 
1 qid:2 1:0 2:0 3:1 4:0.1 5:0 # 2C 
1 qid:2 1:0 2:0 3:1 4:0.2 5:0 # 2D  
2 qid:3 1:0 2:0 3:1 4:0.1 5:1 # 3A 
3 qid:3 1:1 2:1 3:0 4:0.3 5:0 # 3B 
4 qid:3 1:1 2:0 3:0 4:0.4 5:1 # 3C 
1 qid:3 1:0 2:1 3:1 4:0.5 5:0 # 3D

Все написанное после "#" не читается (комментарий).

In [10]:
def read_file(file_path, features_count):
    ''' Считывает дата-файл по адресу file_path, в котором есть не более features_count признаков.
    Возвращает список меток релевантности, id запросов и матрицу признаков'''
    
    relevs = []
    qids = []
    features = []
    
    with open(file_path) as data_file:
        for line in data_file:
            split_line = line.split(' ')
            
            # релевантность и id запроса
            relevs.append(split_line[0])
            qids.append(int(split_line[1].split(':')[1]))
            
            # признаки
            object_features = np.zeros(features_count, dtype=float)
            for feat in split_line[2:]:
                index, value = map(float, feat.split(':'))
                object_features[index] = value
            
            features.append(object_features)
    
    return relevs, qids, np.array(features)


def write(features, relevs, qids, file_path, index_begin, index_end):
    ''' Создает файл по адресу file_path, в который будут записаны релевантности relevs,
    номера запросов qids и признаки features с номера index_begin по index_end. '''
    
    with open(file_path, 'w') as f:
        for index_line in range(index_begin, index_end):
            f.write('{} qid:{}'.format(relevs[index_line], qids[index_line]))
            for i in range(features.shape[1]):
                f.write(' {}:{}'.format(i + 1, features[index_line, i]))
            f.write('\n')
            

def split_to_train_valid_test(relevs, qids, features, 
                              train_path, test_path,
                              train_size, test_size,
                              valid_path=None, valid_size=None):
    ''' Разбивает датасет на две или три части, и записывает их в файлы'''

    num_docs = len(qids)
    i_0 = 0
    
    qids = np.array(qids)
    is_new_query = qids[:-1] != qids[1:]  # True в тех позициях, в которых начинается новый запрос
    new_query_positions = np.arange(num_docs - 1)[is_new_query]  # Позиции, в которых начинается новый запрос

    # Ищем позицию, на которой заканчивается train
    allow_positions = new_query_positions > (train_size * num_docs)
    i_1 = new_query_positions[allow_positions][0] if np.sum(allow_positions) > 0 else num_docs

    # Ищем позицию, на которой заканчивается test
    allow_positions = new_query_positions > ((train_size + test_size) * num_docs)
    i_2 = new_query_positions[allow_positions][0] if np.sum(allow_positions) > 0 else num_docs

    write(features, relevs, qids, train_path, i_0, i_1)
    write(features, relevs, qids, test_path, i_1, i_2)

    if valid_path is not None:
        # Ищем позицию, на которой заканчивается valid
        allow_positions = new_query_positions > ((train_size + test_size + valid_size) * num_docs)
        i_3 = new_query_positions[allow_positions][0] if np.sum(allow_positions) > 0 else num_docs

        write(features, relevs, qids, valid_path, i_2, i_3)

Загрузим датасет

relevs, qids, features = read_file('./data/imat2009_learning.txt', 250)

Разобьем его на три части и запишем каждую в файл. Основная проблема --- для RankLib у объекта должны быть указаны все признаки, даже если они равны нулю. Поэтому приходится их искусственно дописывать.

split_to_train_valid_test(relevs, qids, features,
                          './data/train.txt', './data/test.txt', 0.01, 0.01, 
                          valid_path='./data/valid.txt', valid_size=0.01)

Запустим LambdaMART (бустинг на деревьях, модель 6) на 100 деревьях с 5 листами в каждом. Будем использовать метрику $NDCG_{10}$. Сохраняем саму модель в ./model/LambdaMART_100_5.txt, а вывод обучения в ./model/log_LambdaMART_100_5.txt

In [11]:
%%time
! java -jar RankLib-2.1-patched.jar -train ./data/train.txt -test ./data/test.txt -validate ./data/valid.txt -ranker 6 -tree 100 -leaf 5 -metric2t NDCG@10 -save ./model/LambdaMART_100_5.txt > ./model/log_LambdaMART_100_5.txt

CPU times: user 64.4 ms, sys: 35.1 ms, total: 99.5 ms
Wall time: 3.89 s


1000 деревьев, 10 листев в каждом

In [12]:
%%time
! java -jar RankLib-2.1-patched.jar -train ./data/train.txt -test ./data/test.txt -validate ./data/valid.txt -ranker 6 -metric2t NDCG@10 -save ./model/LambdaMART_1000_10.txt > ./model/log_LambdaMART_1000_10.txt

CPU times: user 181 ms, sys: 80.1 ms, total: 261 ms
Wall time: 11.7 s


Тоже самое, но без валидационной выборки. То есть он будет строить все 1000 деревьев

In [13]:
%%time
! java -jar RankLib-2.1-patched.jar -train ./data/train.txt -test ./data/test.txt -ranker 6 -metric2t NDCG@10 -save ./model/LambdaMART_1000_10_novalid.txt > ./model/log_LambdaMART_1000_10_novalid.txt

CPU times: user 749 ms, sys: 303 ms, total: 1.05 s
Wall time: 42.9 s


Запустим RankNet (нейронная сеть) с различными параметрами

In [12]:
%%time
! java -jar RankLib-2.1-patched.jar -train ./data/train.txt -test ./data/test.txt -validate ./data/valid.txt -ranker 1 -metric2t NDCG@10 -save ./model/RankNet.txt > ./model/log_RankNet.txt

^C
CPU times: user 49.8 ms, sys: 30.1 ms, total: 80 ms
Wall time: 2.86 s


In [13]:
%%time
! java -jar RankLib-2.1-patched.jar -train ./data/train.txt -test ./data/test.txt -validate ./data/valid.txt -ranker 1 -layer 3 -node 30 -metric2t NDCG@10 -save ./model/RankNet_3_30.txt > ./model/log_RankNet_3_30.txt

CPU times: user 1.87 s, sys: 738 ms, total: 2.61 s
Wall time: 1min 40s


# "Игрушечный" поисковый движок

Есть два основный этапа в разработке поискового движка: построение индекса, а затем, используя индекс, ответить на запрос. А затем мы можем добавить результат рейтинга (TF-IDF, PageRank и т.д.), классификацию запрос/документ, и, возможно, немного машинного обучения, чтобы отслеживать последние запросы пользователя и на основе этого выбрать результаты для повышения производительности поисковой системы.

## Построение индекса

Таким образом, первый шаг в построении текстового поискового движка — это сборка перевёрнутого индекса. 
**Перевёрнутый индекс** — это структура данных, которая сопоставляет маркеры с документами, в который они появляются. В данном контексте мы можем рассматривать маркер просто как слова, таким образом перевёрнутый индекс, в своей основе, это что-то, что берёт слово и возвращает нам список документов, где оно встречается.

Необходимо проанализировать и отметить (маркировать, разделив на слова) множество документов. Мы сделаем это следующим образом: для каждого документа, который мы хотим добавить в наш индекс, мы удалим всю пунктуацию и разделить его на пробелы, создадим временную хеш-таблицу, которая соотносит имена документов к списку маркеров.  Вот код, который сделает первоначальную фильтрацию текста:

In [17]:
def process_files(self):
    file_to_terms = {}
    for file in self.filenames:
        pattern = re.compile('[\W_]+')
        file_to_terms[file] = open(file, 'r').read().lower();
        file_to_terms[file] = pattern.sub(' ',file_to_terms[file])
        re.sub(r'[\W_]+','', file_to_terms[file])
        file_to_terms[file] = file_to_terms[file].split()
    return file_to_terms

Перевёрнутый индекс будет картой слова до имени документа, но мы так же хотим поддерживать запросы с фразами: запросы не только для слов, но и для слов в определённой последовательности. Для этого мы должны знать где в документе появляется каждое слово, таким образом мы сможем проверить порядок слов. Я индекс каждого слова в маркированном списке на документ в качестве позиции слова в этом документе, поэтому наш конечный перевернутый индекс будет выглядеть следующим образом:

**{word: {documentID: [pos1, pos2, ...]}, ...}, ...}**

вместо такого:

**{word: [documentID, ...], ...}**


Таким образом, наша первая задача состоит в том, чтобы создать сопоставление слов для своих позиций для каждого документа, а затем объединить их, чтобы создать наш полный перевёрнутый индекс. Это выглядит как:

In [18]:
#input = [word1, word2, ...]
#output = {word1: [pos1, pos2], word2: [pos2, pos434], ...}
def index_one_file(termlist):
    fileIndex = {}
    for index, word in enumerate(termlist):
        if word in fileIndex.keys():
            fileIndex[word].append(index)
        else:
            fileIndex[word] = [index]
    return fileIndex

Код принимает список терминов в документе, разделённые пробелом (в котором слова находятся в их первоначальном порядке), и добавляет каждое в хеш-таблицу, где значением является список позиций этого слова в документе. Мы строим этот список многократно, как мы идём по списку, до тех пор, пока не пройдём все слова, оставив нас с таблицей, снабжённую ключами по строкам и размеченными до списка позиций этих строк.

Теперь нам нужно объединить эти хеш-таблицы. 

Промежуточный формат индекса:

**{documentID: {word: [pos1, pos2, ...]}, ...}**

который преобразуется к окончательному формату:

In [19]:
#input = {filename: [word1, word2, ...], ...}
#res = {filename: {word: [pos1, pos2, ...]}, ...}
def make_indices(termlists):
    total = {}
    for filename in termlists.keys():
        total[filename] = index_one_file(termlists[filename])
    return total

данный код принимает результаты функции file_to_terms, и создает новую хеш-таблицу помеченных ключом по имени файла и со значениями, которые являются результатом предыдущей функции, создавая вложенную хеш-таблицу.

Строим перевернутый индекс:

In [20]:
#input = {filename: {word: [pos1, pos2, ...], ... }}
#res = {word: {filename: [pos1, pos2]}, ...}, ...}
def fullIndex(regdex):
    total_index = {}
    for filename in regdex.keys():
        for word in regdex[filename].keys():
            if word in total_index.keys():
                if filename in total_index[word].keys():
                    total_index[word][filename].extend(regdex[filename][word][:])
                else:
                    total_index[word][filename] = regdex[filename][word]
            else:
                total_index[word] = {filename: regdex[filename][word]}
    return total_index

Теперь мы можем ввести слово, и должны получить перечень документов, в которых оно встречается.

## Выполнение запросов к индексу

Итак, есть два типа запросов, которые мы хотим обрабатывать: стандартные запросы, где по крайней мере одно из слов в запросе появляется в документе и запросы с фразой, где все слова запроса встречаются в документе в том же порядке.

Мы собираемся реализовать стандартные запросы в первую очередь. Простой способ реализовать их — разбить запрос на слова (маркеры, как описано выше), получить список за каждое слово, документы в которых они встречаются, а затем объединить все эти списки. Вот как мы выполним запрос для одного слова:

In [21]:
def one_word_query(self, word):
    pattern = re.compile('[\W_]+')
    word = pattern.sub(' ',word)
    if word in self.invertedIndex.keys():
        return [filename for filename in self.invertedIndex[word].keys()]
    else:
        return []

Теперь стандартный запрос является очень простым расширением. Мы просто агрегируем и объединяем списки как показано здесь:

In [22]:
def free_text_query(self, string):
    pattern = re.compile('[\W_]+')
    string = pattern.sub(' ',string)
    result = []
    for word in string.split():
        result += self.one_word_query(word)
    return list(set(result))

Последним типом запросов является запрос с фразой, который немного сложнее, так как мы должны гарантировать правильный порядок слов в документах.

In [23]:
def phrase_query(self, string):
    pattern = re.compile('[\W_]+')
    string = pattern.sub(' ',string)
    listOfLists, result = [],[]
    for word in string.split():
        listOfLists.append(self.one_word_query(word))
    setted = set(listOfLists[0]).intersection(*listOfLists)
    for filename in setted:
        temp = []
        for word in string.split():
            temp.append(self.invertedIndex[word][filename][:])
        for i in range(len(temp)):
            for ind in range(len(temp[i])):
                temp[i][ind] -= i
        if set(temp[0]).intersection(*temp):
            result.append(filename)
    return self.rankResults(result, string)

И так, мы вновь сначала обрабатываем текст входного запроса. Затем мы запускаем одно слово из запроса для каждого слова на входе, и добавляем каждый из этих списков результатов в наш общий список. Затем мы создаём множество с именем 'setted', который принимает пересечение первого списка со всеми другими списками (по существу, принимая пересечение всех списков), и оставляет нас с промежуточным результатом: множеством всех документов, содержащих все слова запроса.

## Ранжирование

 Заключительным шагом в построении поискового движка является создание системы для ранжирования документов по их релевантности к запросу. Это наиболее сложная часть, поскольку она не имеет прямого технического решения. 

В версии, представленной в папке Text-Search-Engine реализовано TF-IDF ранжирование. Рекомендуется ознакомиться с финальной версией движка.

<h1 align="center"> Conclusion </h1>


** Check questions **

1) Чем отличаются поточечный и попарный подходы к ранжированию? Какой лучше работает и почему?

2) Чем отличается RankSVM от обычного SVM?

3) Что такое коэффициент корреляции Кенделла? Как его использовать для оценки качества модели?

4) Какие алгоритмы ранжирования реализованные в RankLib проекта Lemur мы сегодня рассматривали? 

5) Какие этапы построения простейшего поискового движка можно выделить? В чем их сложность?

## References

[1] C.J.C. Burges, T. Shaked, E. Renshaw, A. Lazier, M. Deeds, N. Hamilton and G. Hullender. Learning to rank using gradient descent. In Proc. of ICML, pages 89-96, 2005.

[2] Y. Freund, R. Iyer, R. Schapire, and Y. Singer. An efficient boosting algorithm for combining preferences. The Journal of Machine Learning Research, 4: 933-969, 2003.

[3] J. Xu and H. Li. AdaRank: a boosting algorithm for information retrieval. In Proc. of SIGIR, pages 391-398, 2007.

[4] D. Metzler and W.B. Croft. Linear feature-based models for information retrieval. Information Retrieval, 10(3): 257-274, 2007.

[5] Q. Wu, C.J.C. Burges, K. Svore and J. Gao. Adapting Boosting for Information Retrieval Measures. Journal of Information Retrieval, 2007.

[6] J.H. Friedman. Greedy function approximation: A gradient boosting machine. Technical Report, IMS Reitz Lecture, Stanford, 1999; see also Annals of Statistics, 2001.

[7] Z. Cao, T. Qin, T.Y. Liu, M. Tsai and H. Li. Learning to Rank: From Pairwise Approach to Listwise Approach. ICML 2007. 

[8] L. Breiman. Random Forests. Machine Learning 45 (1): 5–32, 2001.

** You can find HW7 here** 
  * оцените <a href="https://goo.gl/forms/td2EbpfpAQBQNfme2"> семинар </a>